# 3. Capture patterns with recurrent neural networks

## 3.1 Recurrent neural networks
In the previous module, we have been using rich semantic representations of text, and a simple linear classifier on
top of the embeddings. What this architecture does is to capture aggregated meaning of words in a sentence, but it
does not take into account the order of words, because aggregation operation on top of embeddings removed this
information from the original text. Because these models are unable to model word ordering, they cannot solve
more complex or ambiguous tasks such as text generation or question answering.

To capture the meaning of text sequence, we need to use another neural network architecture, which is called a
**recurrent neural network, or RNN**. In RNN, we pass our sentence through the network one symbol at a time,
and the network produces some state, which we then pass to the network again with the next symbol.

![rnn-model](https://raw.githubusercontent.com/pengfei99/PyTorchTuto/main/notebooks/img/sample-rnn-model-generation.png)

Given the input sequence of tokens X0,...,Xn.  RNN creates a sequence of neural network blocks, and trains this
sequence end-to-end using back propagation. Each network block takes a pair (Xi,Si)as an input, and produces S(i+1)
as a result. Final state Sn or output Xn goes into a linear classifier to produce the result. All network blocks
share the same weights, and are trained end-to-end using one backpropagation pass.

Because state vectors S0,...,Sn are passed through the network, it is able to learn the sequential dependencies
between words. For example, when the word not appears somewhere in the sequence, it can learn to negate certain
elements within the state vector, resulting in negation.

## 3.2 A simple example of RNN

In this example, we will use a simple RNN to classifier the news data set.

In [1]:
import torch
import torchtext
from torchnlp import *
from torchtext.vocab import vocab
from collections import Counter, OrderedDict


In [2]:
# download data
def load_dataset(storage_path):
    print("Loading dataset...")
    train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root=storage_path)
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    return train_dataset, test_dataset


path = "/tmp/pytorch/data"
train, test = load_dataset(path)


Loading dataset...


In [3]:
# build a simple tokenizer
my_tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

# build label class list
label_classes = ['World', 'Sports', 'Business', 'Sci/Tech']


# function that build vocabulary with the token of all text
def build_vocabulary(dataset, tokenizer, ngrams=1, min_freq=1):
    # here we use counter to store the generated token to take in account the token frequency
    counter = Counter()
    # we iterate over all rows, covert text to word token, and add these token to bag_of words
    for (label, line) in dataset:
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line), ngrams=ngrams))
    # sort the collected token counter by token's frequencies
    sorted_by_token_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    # build a set of words as an orderedDict
    words_dict = OrderedDict(sorted_by_token_freq_tuples)
    # we build a vocabulary based on the words token
    return vocab(words_dict, min_freq=min_freq)


# build a vocab
my_vocab = build_vocabulary(train, my_tokenizer)


def encode(text, vocabulary, tokenizer):
    return [vocabulary[word] for word in tokenizer(text)]


my_vocab_size = len(my_vocab)

## 3.2.1 Build a Simple RNN Classifier

In case of simple RNN, each recurrent unit is a simple linear network, which takes
**concatenated input vector and state vector**, and produce a **new state vector**. PyTorch represents this unit
with RNNCell class, and a networks of such cells - as RNN layer.

To define an RNN classifier, we will first apply an embedding layer to lower the dimensionality of input vocabulary,
and then have RNN layer on top of it.

RNNs are quite difficult to train. So keep in mind two important points:
1. **Small learning rate**: because once the RNN cells are unrolled along the sequence length, the
resulting number of layers involved in back propagation is quite large.
2. **Use GPU if you can**: It can take quite a long time to train the network on larger dataset to produce good results.
                     GPU may reduce that training time.

In [4]:
# In this classifier, we will use padded data loader, so each batch will have a number of padded sequences of the
# same length. RNN layer will take the sequence of embedding tensors, and produce two outputs:
# - x: is a sequence of RNN cell outputs at each step
# - h: is a final hidden state for the last element of the sequence
# We then apply a fully-connected linear classifier to get the number of class.
class SimpleRNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        # note our embedding layer is untrained, if you want better results we can use pre-trained embedding layer
        # with Word2Vec or GloVe embeddings, as described in the previous unit. For better understanding, you might
        # want to adapt this code to work with pre-trained embeddings.
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x, h = self.rnn(x)
        return self.fc(x.mean(dim=1))

### 3.2.2 Build a training loop



In [5]:
def select_hardware_for_training(device_name):
    if device_name == 'cpu':
        return 'cpu'
    elif device_name == 'gpu':
        return 'cuda' if (device_name == "") & torch.cuda.is_available() else 'cpu'
    else:
        print("Unknown device name, choose cpu as default device")
        return 'cpu'


device = select_hardware_for_training("cpu")


# define training loop
def train_loop(net, dataloader, lr=0.01, optimizer=None, loss_fn=torch.nn.CrossEntropyLoss(), epoch_size=None,
               report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss, acc, count, i = 0, 0, 0, 0
    for labels, features in dataloader:
        optimizer.zero_grad()
        features, labels = features.to(device), labels.to(device)
        out = net(features)
        loss = loss_fn(out, labels)  #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss += loss
        _, predicted = torch.max(out, 1)
        acc += (predicted == labels).sum()
        count += len(labels)
        i += 1
        if i % report_freq == 0:
            print(f"{count}: acc={acc.item() / count}")
        if epoch_size and count > epoch_size:
            break
    return total_loss.item() / count, acc.item() / count


In [6]:
def padding_text(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label,
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1], my_vocab, my_tokenizer) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len, v))
    return (  # tuple of two tensors - labels and features
        torch.LongTensor([t[0] - 1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t), (0, l - len(t)), mode='constant', value=0) for t in v])
    )


# build a dataloader with text padding tensor
train_loader = torch.utils.data.DataLoader(train, batch_size=16, collate_fn=padding_text, shuffle=True)

In [7]:
simple_rnn_model = SimpleRNNClassifier(my_vocab_size, 64, 32, len(label_classes)).to(device)
train_loop(simple_rnn_model, train_loader, lr=0.001)

3200: acc=0.32
6400: acc=0.3978125
9600: acc=0.47208333333333335
12800: acc=0.525546875
16000: acc=0.5631875
19200: acc=0.5941666666666666
22400: acc=0.6209375
25600: acc=0.64015625
28800: acc=0.6567013888888888
32000: acc=0.672
35200: acc=0.6851420454545455
38400: acc=0.6969270833333333
41600: acc=0.7058413461538462
44800: acc=0.7149107142857143
48000: acc=0.72375
51200: acc=0.73171875
54400: acc=0.73875
57600: acc=0.7451215277777777
60800: acc=0.7511348684210526
64000: acc=0.75590625
67200: acc=0.7608630952380953
70400: acc=0.7652982954545454
73600: acc=0.7694565217391305
76800: acc=0.7738151041666667
80000: acc=0.7781125
83200: acc=0.7820913461538461
86400: acc=0.7852430555555555
89600: acc=0.7884821428571429
92800: acc=0.7910883620689655
96000: acc=0.7937395833333334
99200: acc=0.7963508064516129
102400: acc=0.798994140625
105600: acc=0.8017992424242424
108800: acc=0.8042738970588236
112000: acc=0.8062589285714286
115200: acc=0.8084809027777777
118400: acc=0.8107094594594595


(0.03263011678059896, 0.8118916666666667)

## 3.3 Long Short Term Memory (LSTM) classifier

One of the main problems of classical RNNs is so-called **vanishing gradients** problem. Because RNNs are
trained end-to-end in one back-propagation pass, it is having hard times to back propagate error to the first
layers of the network, and thus the network cannot learn relationships between distant tokens. One of the ways to
avoid this problem is to introduce explicit state management by using so called **gates**. There are two most known
architectures of this kind:
 - Long Short Term Memory (LSTM)
 - Gated Relay Unit (GRU)


![LSTM-model](https://raw.githubusercontent.com/pengfei99/PyTorchTuto/main/notebooks/img/long-short-term-memory-cell.svg)

LSTM Network is organized in a manner similar to RNN, but there are two states that are being passed from layer to
layer:
- **actual state**: c
- **hidden vector**: h

At each unit, hidden vector h(i) is concatenated with input x(i), and they control what happens to the state
c via gates. Each gate is a neural network with sigmoid activation (output in the range [0,1]), which can be
thought of as bitwise mask when multiplied by the state vector. There are the following gates (from left to
right on the figure above):

- forget gate:  takes hidden vector and determines, which components of the vector **c** we need to forget, and which to pass through.
- input gate: takes some information from the input and hidden vector, and inserts it into state.
- output gate: transforms state via some linear layer with tanh activation, then selects some of its components using hidden vector h(i) to produce new state c(i+1).

Components of the state **c** can be thought of as some flags that can be switched on and off. For example, when we
encounter a name Alice in the sequence, we may want to assume that it refers to female character, and raise the flag
in the state that we have female noun in the sentence. When we further encounter phrases and Tom, we will raise the
flag that we have plural noun. Thus by manipulating state we can supposedly keep track of grammatical properties of
sentence parts.

A great resource for understanding internals of LSTM is this great article by Christopher Olah,
Understanding LSTM Networks: https://colah.github.io/posts/2015-08-Understanding-LSTMs/ .

### 3.3.1 Build a LSTM classifier in PyTorch
PyTorch hides the complexe implementation of LSTM inside LSTMCell class, and provides LSTM object to represent the
whole LSTM layer. Thus, implementation of LSTM classifier will be pretty similar to the simple RNN which we have seen
before.

In [8]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data) - 0.5
        self.rnn = torch.nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x, (h, c) = self.rnn(x)
        return self.fc(h[-1])

In [9]:
lstm_model = LSTMClassifier(my_vocab_size, 64, 32, len(label_classes)).to(device)

### 3.3.2 Train the LSTM classifier

Note that training LSTM is also quite slow, and you may not seem much raise in accuracy in the beginning of training.
Also, you may need to play with lr learning rate parameter to find the learning rate that results in reasonable
training speed, and yet does not cause

In [10]:
train_loop(lstm_model, train_loader, lr=0.001)


3200: acc=0.2540625
6400: acc=0.2465625
9600: acc=0.24729166666666666
12800: acc=0.25046875
16000: acc=0.2701875
19200: acc=0.30786458333333333
22400: acc=0.34776785714285713
25600: acc=0.3819921875
28800: acc=0.4117013888888889
32000: acc=0.43503125
35200: acc=0.4556818181818182
38400: acc=0.47265625
41600: acc=0.48677884615384615
44800: acc=0.5008482142857142
48000: acc=0.512875
51200: acc=0.524921875
54400: acc=0.5358823529411765
57600: acc=0.5452430555555555
60800: acc=0.5553289473684211
64000: acc=0.566234375
67200: acc=0.5773660714285714
70400: acc=0.5890767045454546
73600: acc=0.5998913043478261
76800: acc=0.6101432291666666
80000: acc=0.62015
83200: acc=0.6291826923076923
86400: acc=0.6374884259259259
89600: acc=0.6453683035714286
92800: acc=0.6530172413793104
96000: acc=0.66021875
99200: acc=0.6668044354838709
102400: acc=0.67337890625
105600: acc=0.6798484848484848
108800: acc=0.6854319852941176
112000: acc=0.6908482142857143
115200: acc=0.6961024305555555
118400: acc=0.70140

(0.041063944498697914, 0.7039083333333334)

## 3.4 Improve training loop by using packed sequences

In previous training loop, we used dataloader that pad all sequences in the mini-batch with zero vectors. While it
results in some memory waste, with RNNs it is more critical that additional RNN cells are created for the padded
input items, which take part in training, yet do not carry any important input information. It would be much better
to train RNN only to the actual sequence size.

To do that, a special format of padded sequence storage is introduced in PyTorch. Suppose we have input padded
mini-batch which looks like this:

[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]

Here 0 represents padded values, and the actual length vector of input sequences is [5,3,1].

In order to effectively train RNN with padded sequence, we want to begin training first group of RNN cells with
large mini-batch ([1,6,9]), but then end processing of third sequence, and continue training with shorted mini-batches
([2,7], [3,8]), and so on. Thus, packed sequence is represented as one vector - in our case [1,6,9,2,7,3,8,4,5], and
length vector ([5,3,1]), from which we can easily reconstruct the original padded mini-batch.

To use packed sequence, we can use two function:
- **torch.nn.utils.rnn.pack_padded_sequence**: encode a padded tensor to packed sequence tensor.
- **torch.nn.utils.rnn.pad_packed_sequence**: decode a packed sequence tensor to a padded tensor

All recurrent layers in PyTorch including RNN, LSTM and GRU, support packed sequences as input, and produce packed
output, which can be decoded back to padded tensor .

To be able to produce packed sequence, we need to pass length vector to the network, and thus we need a different
collate function in dataloader to prepare mini-batches:

In [18]:
def padding_text_length(b):
    # build vectorized sequence
    v = [encode(x[1],my_vocab,my_tokenizer) for x in b]
    # compute the length of each text sequence and the max length of all text sequence in the mini-batch
    len_seq = list(map(len, v))
    l = max(len_seq)
    return (  # note compare to padding_text function, it returns three tensors, not two.
        # 1st output is label tensors
        torch.LongTensor([t[0] - 1 for t in b]),
        # 2nd output is feature tensors padded with 0.
        torch.stack([torch.nn.functional.pad(torch.tensor(t), (0, l - len(t)), mode='constant', value=0) for t in v]),
        # 3rd output is length of text sequence tensor
        torch.tensor(len_seq)
    )


train_loader_len = torch.utils.data.DataLoader(train, batch_size=16, collate_fn=padding_text_length, shuffle=True)

### 3.4.1 Rewrite LSTM classifier to accept packed dataloader

To use packed dataloader to train the model, we need to modify the LSTMClassifier above. The forward pass will receive
both padded mini-batch and the vector of the text sequence lengths. After computing the embedding, we compute
packed sequence, pass it to LSTM layer, and then unpack the result back.

In [19]:
class LSTMClassifierWithPackedTensor(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data) - 0.5
        self.rnn = torch.nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        # encode padded tensor to packed sequence
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        # pass the packed sequence tensor to lstm layer
        pad_x, (h, c) = self.rnn(pad_x)
        # decode the packed sequence tensor to padded tensor
        # We actually do not return the unpacked result x, because we use output from the hidden layers in the
        # following computations. Thus, we can remove the unpacking altogether from this code. The reason we place
        # it here is for you to be able to modify this code easily, in case you should need to use network output
        # in further computations.
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x, batch_first=True)
        return self.fc(h[-1])

In [20]:
# build a lstm model which accept packed sequence data loader
lstm_packed_seq_model = LSTMClassifierWithPackedTensor(my_vocab_size, 64, 32, len(label_classes)).to(device)

### 3.4.2 Rewrite the training loop for the new LSTM classifier



In [21]:
# define a train loop for offset
def train_loop_packed_seq(net, dataloader, lr=0.01, optimizer=None, loss_fn=torch.nn.CrossEntropyLoss(),
                          epoch_size=None,
                          report_freq=200, use_pack_sequence=False):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss, acc, count, i = 0, 0, 0, 0
    for labels, text, off in dataloader:
        optimizer.zero_grad()
        labels, text = labels.to(device), text.to(device)
        if use_pack_sequence:
            off = off.to('cpu')
        else:
            off = off.to(device)
        out = net(text, off)
        loss = loss_fn(out, labels)  #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss += loss
        _, predicted = torch.max(out, 1)
        acc += (predicted == labels).sum()
        count += len(labels)
        i += 1
        if i % report_freq == 0:
            print(f"{count}: acc={acc.item() / count}")
        if epoch_size and count > epoch_size:
            break
    return total_loss.item() / count, acc.item() / count

In [22]:
train_loop_packed_seq(lstm_packed_seq_model, train_loader_len, lr=0.001, use_pack_sequence=True)

3200: acc=0.3046875
6400: acc=0.3590625
9600: acc=0.40875
12800: acc=0.457265625
16000: acc=0.502625
19200: acc=0.5421354166666666
22400: acc=0.5751339285714285
25600: acc=0.6003125
28800: acc=0.6232291666666666
32000: acc=0.64346875
35200: acc=0.6592045454545454
38400: acc=0.6749739583333333
41600: acc=0.6879086538461539
44800: acc=0.6991294642857143
48000: acc=0.7093958333333333
51200: acc=0.71890625
54400: acc=0.7277573529411765
57600: acc=0.7358333333333333
60800: acc=0.7423684210526316
64000: acc=0.74875
67200: acc=0.7546726190476191
70400: acc=0.7602130681818182
73600: acc=0.7658423913043478
76800: acc=0.7706119791666667
80000: acc=0.7753375
83200: acc=0.7796995192307692
86400: acc=0.7841666666666667
89600: acc=0.7880022321428571
92800: acc=0.7917133620689655
96000: acc=0.79528125
99200: acc=0.7984879032258064
102400: acc=0.80171875
105600: acc=0.8043560606060606
108800: acc=0.80703125
112000: acc=0.8095892857142857
115200: acc=0.8122569444444444
118400: acc=0.8144679054054054


(0.030074466959635417, 0.8154666666666667)

You could notice the accuracy improved a little. Note the structure of the model does not change, we just change the
input tensors format.

## 3.5 Bidirectional and multilayer RNNs

In above examples, all recurrent networks operated in one direction, from beginning of a sequence to the end. It
looks natural, because it resembles the way we read and listen to speech. However, since in many practical cases we
have random access to the input sequence, it might make sense to run recurrent computation in both directions. Such
networks are called **bidirectional RNNs**, and they can be created by passing **bidirectional=True** parameter to
RNN/LSTM/GRU constructor.

When dealing with bidirectional network, we would need two hidden state vectors, one for each direction. PyTorch
encodes those vectors as one vector of twice larger size, which is quite convenient, because you would normally pass
the resulting hidden state to fully-connected linear layer, and you would just need to take this increase in size
into account when creating the layer.

Recurrent network, one-directional or bidirectional, captures certain patterns within a sequence, and can store them
into state vector or pass into output. As with convolutional networks, we can build another recurrent layer on top
of the first one to capture higher level patterns, build from low-level patterns extracted by the first layer. This
leads us to the notion of **multi-layer RNN**, which consists of two or more recurrent networks, where output of
the previous layer is passed to the next layer as input.


![multi-layer-lstm](https://raw.githubusercontent.com/pengfei99/PyTorchTuto/main/notebooks/img/multi-layer-lstm.jpg)

For more detail of multi layer lstm, please visit https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3

PyTorch makes constructing such networks an easy task, because you just need to pass num_layers parameter to 
RNN/LSTM/GRU constructor to build several layers of recurrence automatically. This would also mean that the size of 
hidden/state vector would increase proportionally, and you would need to take this into account when handling the 
output of recurrent layers.


In [ ]:
# todo